# CAF Slice

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from caf_verilog.caf_slice import caf_slice_dot
from caf_verilog.sim_helper import sim_shift
from caf_verilog.quantizer import quantize
from caf_verilog.xcorr import gen_tb_values
from gps_helper.prn import PRN
import sk_dsp_comm.sigsys as ss

## Time Offsets with the same Frequency Offset

Produce the expected test results here to compare first.

In [ ]:
fs = 625e3
f_shift = 20e3
freq_res = 10
n_bits = 8
center = 450
corr_length = 250
default_shift = 0
half_length = corr_length / 2
shift_range = int(half_length)

In [ ]:
def generate_test_signals(time_shift, freq_shift, f_samp):
    prn = PRN(10)
    fs = f_samp
    Ns = 5
    prn_seq = prn.prn_seq()
    prn_seq, b = ss.nrz_bits2(np.array(prn_seq), Ns)
    prn_seq = [*prn_seq, *prn_seq]
    prn_seq = np.array(prn_seq)
    prn_seq = prn_seq + prn_seq * 1j
    ref, rec = sim_shift(prn_seq, center, corr_length, shift=time_shift, freq_shift=freq_shift, fs=fs)
    ref_quant = quantize(ref, n_bits)
    rec_quant = quantize(rec, n_bits)
    return ref_quant, rec_quant

In [ ]:
output_cap = []
for shift_in_range in range(-1 * shift_range, shift_range + 1, 5):
    ref_quant, rec_quant = generate_test_signals(time_shift=shift_in_range, freq_shift=f_shift, f_samp=fs)
    assert len(ref_quant) == corr_length
    assert len(rec_quant) == corr_length * 2
    out = caf_slice_dot(ref_quant, rec_quant, f_shift, fs, n_bits=n_bits)
    output_cap.append(out)

In [ ]:
plt_vals = np.array(output_cap[25])
plt_vals = np.abs(plt_vals)
plt.plot(plt_vals)
plt.grid();
np.argmax(plt_vals)

In [ ]:
index_verifications = []
max_vals = []
for dot_arr in output_cap:
    da_np = np.array(dot_arr)
    plt_vals = np.abs(da_np)
    np_am = np.argmax(plt_vals)
    index_verifications.append(np_am)
    max_vals.append(np.abs(dot_arr[np_am]))

In [ ]:
plt.plot(max_vals)
plt.grid();

In [ ]:
!ls ./caf_slice_v/

In [ ]:
time_shifts_status_df = pd.read_csv('caf_slice_v/time_shifts_status_file.csv')

In [ ]:
time_shifts_status_df

In [ ]:
plt.plot(time_shifts_status_df['time_shift'], time_shifts_status_df['out_max'])
plt.grid();

## Frequency Matching with No Time Offset

In [ ]:
freq_match_status_df = pd.read_csv('caf_slice_v/freq_match_status_file.csv')

In [ ]:
output_cap = []
for f_shift_to_match in freq_match_status_df['frequency_shift']:
    ref_quant, rec_quant = generate_test_signals(time_shift=0, freq_shift=f_shift_to_match, f_samp=fs)
    assert len(ref_quant) == corr_length
    assert len(rec_quant) == corr_length * 2
    out = caf_slice_dot(ref_quant, rec_quant, f_shift_to_match, fs)
    output_cap.append(out)

In [ ]:
index_verifications = []
max_vals = []
for dot_arr in output_cap:
    da_np = np.array(dot_arr)
    plt_vals = np.abs(da_np)
    np_am = np.argmax(plt_vals)
    index_verifications.append(np_am)
    max_vals.append(np.abs(dot_arr[np_am]))

In [ ]:
plt.plot(max_vals)
plt.grid();

In [ ]:
output_cap = []
for f_shift_to_match in freq_match_status_df['frequency_shift']:
    ref_quant, rec_quant = generate_test_signals(time_shift=0, freq_shift=f_shift_to_match, f_samp=fs)
    assert len(ref_quant) == corr_length
    assert len(rec_quant) == corr_length * 2
    out = caf_slice_dot(ref_quant, rec_quant, f_shift_to_match, fs, n_bits=8)
    output_cap.append(out)

In [ ]:
index_verifications = []
max_vals = []
for dot_arr in output_cap:
    da_np = np.array(dot_arr)
    plt_vals = np.abs(da_np)
    np_am = np.argmax(plt_vals)
    index_verifications.append(np_am)
    max_vals.append(np.abs(dot_arr[np_am]))

In [ ]:
plt.plot(max_vals)
plt.grid();

In [ ]:
freq_match_status_df

In [ ]:
plt.plot(freq_match_status_df['frequency_shift'], freq_match_status_df['out_max'])
plt.grid();